In [ ]:
import cv2
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource

def fundus_roi(image, mask=None):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    threshold = 15
    _, roi = cv2.threshold(v, threshold, 1, cv2.THRESH_BINARY)
    roi = roi.astype(np.uint8)
    white_pixels = np.argwhere(roi == 1)
    if white_pixels.size == 0:
        print("Aucun pixel blanc trouvé dans le masque.")
        return {"roi": roi, "diameter": 0, "image": image}
    x_min, y_min = np.min(white_pixels, axis=0)
    x_max, y_max = np.max(white_pixels, axis=0)
    diameter_x = x_max - x_min
    diameter_y = y_max - y_min
    diameter = int(np.maximum(diameter_x, diameter_y))
    return {"roi": roi, "diameter": diameter, "image": image}

def apply_clahe(image, diameter=None, roi=None):
    if diameter is None:
        print('Erreur: Le diamètre est None. Impossible de continuer.')
        return None
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    if len(image.shape) == 2:  # Grayscale image
        return {"image": clahe.apply(image), "diameter": diameter}
    elif len(image.shape) == 3 and image.shape[2] == 3:  # Color image
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv_planes = list(cv2.split(hsv_image))  # Convert to list to allow modification
        hsv_planes[2] = clahe.apply(hsv_planes[2])
        return {"image": cv2.cvtColor(cv2.merge(hsv_planes), cv2.COLOR_HSV2BGR), "diameter": diameter}
    else:
        raise ValueError("Unsupported image format for CLAHE")

def apply_sarki(image, diameter=None, roi=None):
    data = fundus_roi(image)
    clahe_image = apply_clahe(**data)

    hsv_clahe_image = cv2.cvtColor(clahe_image["image"], cv2.COLOR_BGR2HSV)
    hsv_float = hsv_clahe_image.astype(np.float32) / 255.0
    mu0 = 0.5  # Intensity target
    d1 = int(clahe_image["diameter"] / 15)
    if d1 % 2 == 0:
        d1 += 1
    muL = cv2.blur(hsv_float[:, :, 2], (d1, d1))  # Local mean

    # Processing pixel value
    v_processed = hsv_float[:, :, 2] + mu0 - muL
    v_processed = np.clip(v_processed, 0, 1)
    hsv_float[:, :, 2] = v_processed
    
    hsv_uint8 = (hsv_float * 255).astype(np.uint8)
    final_image = cv2.cvtColor(hsv_uint8, cv2.COLOR_HSV2BGR)

    return final_image

# Load the image
file_in = '../data/image_1.jpg'
image = cv2.imread(file_in)

# Apply preprocessing
preprocessed_image = apply_sarki(image)

# Convert images for Bokeh display
def cvt2bokeh(img):
    h, w = img.shape[:2]
    container = np.empty((h, w), dtype=np.uint32)
    view = container.view(dtype=np.uint8).reshape((h, w, 4))
    view[:, :, 0] = img[::-1, :, 0]  # copy red channel
    view[:, :, 1] = img[::-1, :, 1]  # copy blue channel
    view[:, :, 2] = img[::-1, :, 2]  # copy green channel
    view[:, :, 3] = 255
    return container

image = cvt2bokeh(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

preprocessed_bokeh = cvt2bokeh(cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2RGB))

# Setup Bokeh output
output_notebook()

# Create ColumnDataSource for Bokeh
source1 = ColumnDataSource(data=dict(image=[image]))
source2 = ColumnDataSource(data=dict(image=[preprocessed_bokeh]))

# Calculate image dimensions
image_height, image_width = image.shape[:2]

p1 = figure(title="Original Image", x_range=(0, image_width), y_range=(0, image_height),
            width=image_width//10, height=image_height//10)
p1.image_rgba(image='image', source=source1, x=0, y=0, dw=image_width, dh=image_height)


# Create Bokeh figure
p2 = figure(title="Final Preprocessed Image", 
           x_range=(0, image_width), y_range=(0, image_height),
           width=image_width//10, height=image_height//10)
p2.image_rgba(image='image', source=source2, x=0, y=0, dw=image_width, dh=image_height)

p = gridplot([[p1, p2]])

# Show figure
show(p)
